In [2]:
!pip install transformers
!pip install question_generation
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.7 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 596 kB 87.1 MB/s 
     |████████████████████████████████| 6.6 MB 69.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 22.3 MB/s 
     |████████████████████████████████| 212 kB 77.4 MB/s 
     |████████████████████████████████| 1.1 MB 68.3 MB/s 
     |████████████████████████████████| 140 kB 71.6 MB/s 
     |████████████████████████████████| 127 kB 67.9 MB/s 
     |████████████████████████████████| 271 kB 69.9 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |███████████

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Yangdf/mt5-base-chinese-qg")

model = AutoModelForSeq2SeqLM.from_pretrained("Yangdf/mt5-base-chinese-qg")

from question_generation import pipeline

# gpu版本
qg = pipeline("question-generation", device="cuda") 
qa = pipeline("question-answering", device="cuda") 

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

## 处理 CSV 格式的数据

In [ ]:
import pandas as pd
import glob
import json
import csv
import re
import os

path = r'drive/MyDrive/Database/extract_results/20220615/kmcha' # use your path

all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    test_list = []
    # 提取需要生成问题的文件名
    fname = filename.split('/')[-1:][0]
    fname = os.path.splitext(fname)[0]
    # 遍历词条信息进行问题生成
    with open(filename, 'r', encoding='utf-8-sig') as csvfile:
        csvReader = csv.DictReader(csvfile)
        i = 0
        for row in csvReader:
            y = {
                '_id':'',
                'text':'',
                'qa_pair':[]}
            i = i + 1
            full_text = row['line']
            # 正则表达式，去除[]、()中的附加文本内容，减少对模型生成问题的无关影响
            full_text = re.sub("[\(\[].*?[\)\]]", "", full_text)
            # 对row进行转换，把原本的OrderedDict转换成string，此处因为保存原格式模型的表现更好，故此处进行保留
            # row = str(dict(row))
            print('FullText: ', full_text)
            y['_id'] = i
            questions = qg(full_text)
            j = 0
            for q in questions:
                z = {
                     'c_id':'',
                     'q':'',
                     'a':'',
                     'label':''
                }
                y['text'] = full_text
                answer = qa({
                    'question': str(q),
                    'context': full_text,
                })
                print('Q: ', q)
                # print('Row: ', row)
                if bool(answer) is False:
                    print('{ }')
                else:
                    j = j + 1
                    z['c_id'] = j
                    z['q'] = q
                    z['a'] = answer['answer']
                    print('Answer: ', answer['answer'])
                    print('\n')
                    y['qa_pair'].append(z)
            # 在满足q_a pair不空的情况下才进行添加
            if len(y['qa_pair']) == 0:
                pass
            else:
                test_list.append(y)
    with open('drive/MyDrive/Database/test_data/20220617/'+fname+'.json', 'a', encoding='utf-8-sig') as file:
        json.dump(test_list, file, ensure_ascii=False) 

FullText:  小年，农历十二月为29天的年份。。
Q:  小年是哪一天
Answer:  农历十二月


Q:  小年是什么时候
Answer:  农历十二月


Q:  小年的是哪一天
Answer:  农历十二月


FullText:  孟兰节，孟兰节 - 百科全书。盂兰实际是佛教的节日，"盂兰”为梵文，意为救倒悬、解痛苦。在农历七月十五日，在香港、东南亚一带，盂兰节相传阴间地府的鬼门关会大开，无数鬼魂涌到阳间流连，可谓“阴气冲天”。
Q:  阴间地府的鬼门关会大开,鬼魂涌到什么地方流连?
Answer:  阳间


Q:  什么节日是佛教的节日?
Answer:  孟兰节


Q:  在农历七月十五日,在香港、东南亚一带,什么节日被称为“阴气冲天”?
Answer:  孟兰节


FullText:  立春，二十四节气之一，在2月3、4或5日。我国以立春为春季的开始。见〖节气〗、。
Q:  什么节气是立春的开始?
{ }
Q:  立春是什么时候开始的?
Answer:  2月3、4或5日


Q:  什么节气的开始是立春?
{ }
FullText:  惊蛰，节气名。在每年公历3月6日前后。冬眠动物将四处活动。渐有春雷。。
Q:  惊蛰的节气名是什么?
Answer:  惊蛰


Q:  惊蛰节气名是什么时候?
Answer:  3月6日前后


Q:  春雷的节气是什么?
Answer:  惊蛰


FullText:  立夏，二十四节气之一，在5月5、6或7日。我国以立夏为夏季的开始。见〖节气〗、。
Q:  什么节气是立夏的开始?
{ }
Q:  立夏是什么时候开始的?
Answer:  5月5、6或7日


Q:  什么节气的开始是立夏?
{ }
FullText:  火把节，白、彝、傈僳、纳西、拉祜等民族的传统节日。一般在农历六月二十四日。这一天，各村寨中竖起大火把，各家门上插着小火把。入夜点燃，人们举着火把奔跑在田间，表示驱除虫害。此外，人们盛装相贺，进行斗牛和其他娱乐活动。。
Q:  火把节在白、彝、傈僳、纳西、拉祜等民族的传统节日是什么?
{ }
Q:  在火把节中,人们举着火把奔跑在田间,驱除虫害的什么?
{ }
Q:  火把节日的日期是什么时候?
Answer:  农历六月二十四日


FullText:  腊八节，腊八节 - 百科

## 处理 txt 格式的数据

In [5]:
import pandas as pd
import glob
import json
import csv
import re
import os

path = r'drive/MyDrive/Database/extract_results/20220622/wiki' # use your path

all_files = glob.glob(path + "/*.txt")

for filename in all_files:
    test_list = []
    # 提取需要生成问题的文件名
    fname = filename.split('/')[-1:][0]
    fname = os.path.splitext(fname)[0]
    # 遍历词条信息进行问题生成
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        i = 0
        for row in lines:
            y = {
                '_id':'',
                'text':'',
                'qa_pair':[]}
            i = i + 1
            full_text = row
            print('Full text: ', full_text)
            # 正则表达式，去除[]、()中的附加文本内容，减少对模型生成问题的无关影响
            full_text = re.sub("[\(\[].*?[\)\]]", "", full_text)
            # 对row进行转换，把原本的OrderedDict转换成string，此处因为保存原格式模型的表现更好，故此处进行保留
            # row = str(dict(row))
            print('FullText: ', full_text)
            y['_id'] = i
            questions = qg(full_text)
            j = 0
            for q in questions:
                z = {
                     'c_id':'',
                     'q':'',
                     'a':'',
                     'label':''
                }
                y['text'] = full_text
                answer = qa({
                    'question': str(q),
                    'context': full_text,
                })
                print('Q: ', q)
                # print('Row: ', row)
                if bool(answer) is False:
                    print('{ }')
                else:
                    j = j + 1
                    z['c_id'] = j
                    z['q'] = q
                    z['a'] = answer['answer']
                    print('Answer: ', answer['answer'])
                    print('\n')
                    y['qa_pair'].append(z)
            # 在满足q_a pair不空的情况下才进行添加
            if len(y['qa_pair']) == 0:
                pass
            else:
                test_list.append(y)
    with open('drive/MyDrive/Database/test_data/20220622/wiki/'+fname+'.json', 'a', encoding='utf-8-sig') as file:
        json.dump(test_list, file, ensure_ascii=False) 

Streaming output truncated to the last 5000 lines.
Q:  温庭筠为什么号“温锺馗”?
Answer:  由于形貌奇丑


Q:  为什么温庭筠被温造收养?
Answer:  早年丧父


Q:  谁曾央求温庭筠代笔二十首《菩萨蛮》词?
Answer:  令狐绹


Full text:  李频（？－876年），字德新，睦州寿昌（今浙江建德李家镇石门堂村）人，唐代政治人物、诗人。

FullText:  李频（？－876年），字德新，睦州寿昌（今浙江建德李家镇石门堂村）人，唐代政治人物、诗人。

Q:  李频的字是什么?
Answer:  德新


Q:  李频的职业是什么?
Answer:  诗人


Q:  唐代政治人物、诗人是谁?
Answer:  李频


Full text:  幼颖悟，博学强记[1]，大中八年（854年）甲戌科中进士，任武功县令，曾修复六门堰。后任侍御史。升任都官员外郎。不久，任建州(今福建建瓯)刺史。受百姓爱戴。干符三年（876年），卒于任上，建州举城致哀，建梨岳庙以祀之。李频能诗，然其一生历经战乱，作品多散佚。有诗集《梨岳集》。

FullText:  幼颖悟，博学强记，大中八年（854年）甲戌科中进士，任武功县令，曾修复六门堰。后任侍御史。升任都官员外郎。不久，任建州刺史。受百姓爱戴。干符三年（876年），卒于任上，建州举城致哀，建梨岳庙以祀之。李频能诗，然其一生历经战乱，作品多散佚。有诗集《梨岳集》。

Q:  李频能诗,然其一生历经战乱,作品多散佚?
{ }
Q:  幼颖悟的诗集《梨岳集》是谁写的?
Answer:  李频


Q:  李频有什么诗集?
Answer:  《梨岳集》


Full text:  曾祖郑邻、祖父郑穆皆进士及第。外祖父李翱。其父郑亚更连中进士、贤良方正、书判拔萃三科，官桂管观察使。宝历元年（825年）郑畋生于桂州，小字桂儿。[1]会昌二年（842年）中进士，仕藩镇幕府。咸通五年（864年）进入朝廷，累次升官至中书舍人。十年，迁户部侍郎。十一年，充翰林学士。后来因在起草贬刘瞻的诏书时名贬实褒，被宰相路岩贬为梧州刺史。

FullText:  曾祖郑邻、祖父郑穆皆进士及第。外祖父李翱。其父郑亚更连中进士、贤良方正、书判拔萃三科，官桂管观察使。宝历元年（82